In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
pd.options.display.float_format = '{:.2f}'.format

In [2]:
# Read in New York Times online csv file with daily COVID data
# Convert FIPS number to integer, convert date to datetime, save file to local csv

us_covid_df = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/master/us-counties.csv')
us_covid_df['fips'] = us_covid_df['fips'].fillna(0).astype(np.int64)
us_covid_df['date'] = pd.to_datetime(us_covid_df['date'], yearfirst=True)
us_covid_df.to_csv('us_covid.csv', index=False)

In [3]:
# Read in national population data provided by state, county, FIPS

us_pops_df = pd.read_csv('US_Populations.csv', engine='python', thousands=',')
us_pops_df

,FIPStxt,State,State2,Area_Name,Region,population
0,2000,Alaska,AK,Alaska,State,731545
1,2013,Alaska,AK,Aleutians East Borough,County,3337
2,2016,Alaska,AK,Aleutians West Census Area,County,5634
3,2020,Alaska,AK,Anchorage,County,288000
4,2050,Alaska,AK,Bethel Census Area,County,18386
...,...,...,...,...,...,...
3146,22119,Louisiana,LA,Webster,County,38340
3147,22121,Louisiana,LA,West Baton Rouge,County,26465
3148,22123,Louisiana,LA,West Carroll,County,10830
3149,22125,Louisiana,LA,West Feliciana,County,15568


In [4]:
# Merge New York Times data with population data

us_data_df = pd.merge(us_covid_df, us_pops_df, left_on=['county', 'state'], right_on=['Area_Name', 'State'])
us_data_df

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
0,2020-01-21,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
1,2020-01-22,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
2,2020-01-23,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
3,2020-01-24,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
4,2020-01-25,Snohomish,Washington,53061,1,0.00,53061,Washington,WA,Snohomish,County,822083
...,...,...,...,...,...,...,...,...,...,...,...,...
1066625,2021-03-08,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1066626,2021-03-09,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1066627,2021-03-10,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86
1066628,2021-03-11,Kalawao,Hawaii,15005,1,0.00,15005,Hawaii,HI,Kalawao,County,86


In [5]:
# Separate out data from the state level

states = us_data_df[(us_data_df['Region'] == 'State')]
states

,date,county,state,fips,cases,deaths,FIPStxt,State,State2,Area_Name,Region,population
28839,2020-03-07,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28841,2020-03-08,District of Columbia,District of Columbia,11001,1,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28843,2020-03-09,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28845,2020-03-10,District of Columbia,District of Columbia,11001,4,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
28847,2020-03-11,District of Columbia,District of Columbia,11001,10,0.00,11000,District of Columbia,DC,District of Columbia,State,705749
...,...,...,...,...,...,...,...,...,...,...,...,...
659772,2021-03-08,Arkansas,Arkansas,5001,2005,33.00,5000,Arkansas,AR,Arkansas,State,3017804
659774,2021-03-09,Arkansas,Arkansas,5001,2005,33.00,5000,Arkansas,AR,Arkansas,State,3017804
659776,2021-03-10,Arkansas,Arkansas,5001,2006,33.00,5000,Arkansas,AR,Arkansas,State,3017804
659778,2021-03-11,Arkansas,Arkansas,5001,2006,33.00,5000,Arkansas,AR,Arkansas,State,3017804


In [6]:
# Separate out data specific to county level, drop redundant columns
#  Create infection and death rates, updating for each date in the dataframe

counties = us_data_df.drop(['fips', 'State','Area_Name'], axis=1)
counties['current_crate'] = counties['cases']*100000/counties['population']
counties['current_drate'] = counties['deaths']*100000/counties['population']
counties = counties[(counties['Region'] == 'County')]
counties

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00
...,...,...,...,...,...,...,...,...,...,...,...
1066625,2021-03-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1066626,2021-03-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1066627,2021-03-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00
1066628,2021-03-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00


In [7]:
# Isolate data from most current day.

counties_us = counties[(counties['date'] == '2021-03-12') & (counties['Region'] == 'County')]
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate
416,2021-03-12,Snohomish,Washington,31617,548.00,53061,WA,County,822083,3845.96,66.66
830,2021-03-12,Cook,Illinois,481787,10051.00,17031,IL,County,5150233,9354.66,195.16
1243,2021-03-12,Orange,California,263471,4408.00,6059,CA,County,3175692,8296.49,138.80
1655,2021-03-12,Maricopa,Arizona,520291,9428.00,4013,AZ,County,4485414,11599.62,210.19
2067,2021-03-12,Los Angeles,California,1208678,22304.00,6037,CA,County,10039107,12039.70,222.17
...,...,...,...,...,...,...,...,...,...,...,...
1066149,2021-03-12,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08
1066300,2021-03-12,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00
1066420,2021-03-12,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00
1066536,2021-03-12,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00


In [8]:
# Create up to date rates so rankings can be performed using the latest data.

counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)
counties_us = counties_us[(counties_us['Region'] == 'County')]
counties_us

<ipython-input-8-608f6ade14d5>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['drate'] = counties_us['deaths']/(counties_us['population']/100000)
<ipython-input-8-608f6ade14d5>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_us['crate'] = counties_us['cases']/(counties_us['population']/100000)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate
416,2021-03-12,Snohomish,Washington,31617,548.00,53061,WA,County,822083,3845.96,66.66,66.66,3845.96
830,2021-03-12,Cook,Illinois,481787,10051.00,17031,IL,County,5150233,9354.66,195.16,195.16,9354.66
1243,2021-03-12,Orange,California,263471,4408.00,6059,CA,County,3175692,8296.49,138.80,138.80,8296.49
1655,2021-03-12,Maricopa,Arizona,520291,9428.00,4013,AZ,County,4485414,11599.62,210.19,210.19,11599.62
2067,2021-03-12,Los Angeles,California,1208678,22304.00,6037,CA,County,10039107,12039.70,222.17,222.17,12039.70
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1066149,2021-03-12,Wheeler,Oregon,22,1.00,41069,OR,County,1332,1651.65,75.08,75.08,1651.65
1066300,2021-03-12,King,Texas,11,0.00,48269,TX,County,272,4044.12,0.00,0.00,4044.12
1066420,2021-03-12,Esmeralda,Nevada,37,0.00,32009,NV,County,873,4238.26,0.00,0.00,4238.26
1066536,2021-03-12,Loving,Texas,1,0.00,48301,TX,County,169,591.72,0.00,0.00,591.72


In [9]:
# Create ranking of infection rates by county for each state

counties_us.sort_values(by=['state', 'crate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_case']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case
750611,2021-03-12,Clarke,Alabama,3460,55.00,1025,AL,County,23622,14647.36,232.83,232.83,14647.36,1
689791,2021-03-12,Hale,Alabama,2130,72.00,1065,AL,County,14651,14538.26,491.43,491.43,14538.26,2
540081,2021-03-12,Lowndes,Alabama,1338,52.00,1085,AL,County,9726,13756.94,534.65,534.65,13756.94,3
479018,2021-03-12,Etowah,Alabama,13470,335.00,1055,AL,County,102268,13171.28,327.57,327.57,13171.28,4
540433,2021-03-12,Marengo,Alabama,2464,59.00,1091,AL,County,18863,13062.61,312.78,312.78,13062.61,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
658383,2021-03-12,Converse,Wyoming,972,17.00,56009,WY,County,13822,7032.27,122.99,122.99,7032.27,19
658733,2021-03-12,Sublette,Wyoming,681,7.00,56035,WY,County,9831,6927.07,71.20,71.20,6927.07,20
817888,2021-03-12,Lincoln,Wyoming,1278,12.00,56023,WY,County,19830,6444.78,60.51,60.51,6444.78,21
909774,2021-03-12,Niobrara,Wyoming,150,2.00,56027,WY,County,2356,6366.72,84.89,84.89,6366.72,22


In [10]:
# Create ranking of death rates by county for each state

counties_us.sort_values(by=['state', 'drate', 'county'], inplace=True, ascending=(True, False, True))
counties_us['state_death']=counties_us.groupby([counties_us.State2]).cumcount()+1
counties_us

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_case,state_death
540081,2021-03-12,Lowndes,Alabama,1338,52.00,1085,AL,County,9726,13756.94,534.65,534.65,13756.94,3,1
689791,2021-03-12,Hale,Alabama,2130,72.00,1065,AL,County,14651,14538.26,491.43,491.43,14538.26,2,2
230129,2021-03-12,Walker,Alabama,6679,267.00,1127,AL,County,63521,10514.63,420.33,420.33,10514.63,31,3
477606,2021-03-12,Clay,Alabama,1467,54.00,1027,AL,County,13235,11084.25,408.01,408.01,11084.25,20,4
539377,2021-03-12,Crenshaw,Alabama,1484,55.00,1041,AL,County,13772,10775.49,399.36,399.36,10775.49,29,5
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
580913,2021-03-12,Hot Springs,Wyoming,362,3.00,56017,WY,County,4413,8203.04,67.98,67.98,8203.04,17,19
817888,2021-03-12,Lincoln,Wyoming,1278,12.00,56023,WY,County,19830,6444.78,60.51,60.51,6444.78,21,20
797592,2021-03-12,Uinta,Wyoming,2088,12.00,56041,WY,County,20226,10323.35,59.33,59.33,10323.35,4,21
229052,2021-03-12,Teton,Wyoming,3427,9.00,56039,WY,County,23464,14605.35,38.36,38.36,14605.35,1,22


In [11]:
# Merge data from today with historical data to provide current rates + rates from previous points in time.

usa = pd.merge(counties, counties_us[['state', 'county','drate', 'crate', 'state_death', 'state_case']], on=['state', 'county'])
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case
0,2020-01-21,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
1,2020-01-22,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
2,2020-01-23,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
3,2020-01-24,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
4,2020-01-25,Snohomish,Washington,1,0.00,53061,WA,County,822083,0.12,0.00,66.66,3845.96,15,21
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1064112,2021-03-08,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1064113,2021-03-09,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1064114,2021-03-10,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4
1064115,2021-03-11,Kalawao,Hawaii,1,0.00,15005,HI,County,86,1162.79,0.00,0.00,1162.79,5,4


In [12]:
# Break out daily case data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_case', 'date'], inplace=True)
usa['case_day'] = usa.groupby(['state','county']).cases.diff().fillna(usa.cases)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day
747753,2020-04-01,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,232.83,14647.36,37,1,2.00
747754,2020-04-02,Clarke,Alabama,2,0.00,1025,AL,County,23622,8.47,0.00,232.83,14647.36,37,1,0.00
747755,2020-04-03,Clarke,Alabama,3,0.00,1025,AL,County,23622,12.70,0.00,232.83,14647.36,37,1,1.00
747756,2020-04-04,Clarke,Alabama,7,0.00,1025,AL,County,23622,29.63,0.00,232.83,14647.36,37,1,4.00
747757,2020-04-05,Clarke,Alabama,9,0.00,1025,AL,County,23622,38.10,0.00,232.83,14647.36,37,1,2.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
831492,2021-03-08,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
831493,2021-03-09,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
831494,2021-03-10,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00
831495,2021-03-11,Crook,Wyoming,418,11.00,56011,WY,County,7584,5511.60,145.04,145.04,5511.60,7,23,0.00


In [13]:
# Break out daily death data from the New York Times csv that is cumulative in nature

usa.sort_values(by=['state', 'state_death', 'date'], inplace=True)
usa['death_day'] = usa.groupby(['state','county']).deaths.diff().fillna(usa.deaths)
usa

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
537917,2020-03-26,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13756.94,1,3,1.00,0.00
537918,2020-03-27,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13756.94,1,3,0.00,0.00
537919,2020-03-28,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13756.94,1,3,0.00,0.00
537920,2020-03-29,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13756.94,1,3,0.00,0.00
537921,2020-03-30,Lowndes,Alabama,1,0.00,1085,AL,County,9726,10.28,0.00,534.65,13756.94,1,3,0.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
536152,2021-03-08,Albany,Wyoming,3917,11.00,56001,WY,County,38880,10074.59,28.29,28.29,10095.16,23,6,4.00,0.00
536153,2021-03-09,Albany,Wyoming,3920,11.00,56001,WY,County,38880,10082.30,28.29,28.29,10095.16,23,6,3.00,0.00
536154,2021-03-10,Albany,Wyoming,3920,11.00,56001,WY,County,38880,10082.30,28.29,28.29,10095.16,23,6,0.00,0.00
536155,2021-03-11,Albany,Wyoming,3922,11.00,56001,WY,County,38880,10087.45,28.29,28.29,10095.16,23,6,2.00,0.00


In [14]:
# Export updated national data to csv for use in Tableau

usa.to_csv('daily_covid_us.csv', index=False)

In [15]:
# Filter California data to show only numbers for today

counties_cal = usa[(usa['date'] == '2021-03-12') & (usa['state'] == 'California')]
counties_cal

,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
274613,2021-03-12,Imperial,California,27045,677.00,6025,CA,County,181215,14924.26,373.59,373.59,14924.26,1,2,-6.00,2.00
2067,2021-03-12,Los Angeles,California,1208678,22304.00,6037,CA,County,10039107,12039.70,222.17,222.17,12039.70,2,5,6.00,0.00
480382,2021-03-12,Inyo,California,1335,37.00,6027,CA,County,18039,7400.63,205.11,205.11,7400.63,3,25,7.00,0.00
61325,2021-03-12,Stanislaus,California,57373,966.00,6099,CA,County,550660,10418.95,175.43,175.43,10418.95,4,9,153.00,3.00
76002,2021-03-12,Tulare,California,48550,784.00,6107,CA,County,466195,10414.10,168.17,168.17,10414.10,5,10,71.00,2.00
28467,2021-03-12,Riverside,California,291866,3971.00,6065,CA,County,2470546,11813.83,160.73,160.73,11813.83,6,6,191.00,30.00
48815,2021-03-12,San Joaquin,California,67817,1210.00,6077,CA,County,762148,8898.14,158.76,158.76,8898.14,7,18,0.00,0.00
582260,2021-03-12,Kings,California,22320,235.00,6031,CA,County,152940,14593.96,153.66,153.66,14593.96,8,3,50.00,2.00
27725,2021-03-12,Fresno,California,97026,1527.00,6019,CA,County,999101,9711.33,152.84,152.84,9711.33,9,14,160.00,8.00
435441,2021-03-12,Merced,California,29796,421.00,6047,CA,County,277680,10730.34,151.61,151.61,10730.34,10,8,85.00,3.00


In [16]:
# Show top 10 California counties based on largest current infection rate

counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)
top10_ca_case_rates = counties_cal.head(10)
top10_ca_case_rates

<ipython-input-16-94608615179b>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['crate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
1013074,2021-03-12,Lassen,California,5629,24.00,6035,CA,County,30573,18411.67,78.50,78.50,18411.67,31,1,0.00,0.00
274613,2021-03-12,Imperial,California,27045,677.00,6025,CA,County,181215,14924.26,373.59,373.59,14924.26,1,2,-6.00,2.00
582260,2021-03-12,Kings,California,22320,235.00,6031,CA,County,152940,14593.96,153.66,153.66,14593.96,8,3,50.00,2.00
136933,2021-03-12,San Bernardino,California,288364,3282.00,6071,CA,County,2180085,13227.19,150.54,150.54,13227.19,11,4,67.00,57.00
2067,2021-03-12,Los Angeles,California,1208678,22304.00,6037,CA,County,10039107,12039.70,222.17,222.17,12039.70,2,5,6.00,0.00
28467,2021-03-12,Riverside,California,291866,3971.00,6065,CA,County,2470546,11813.83,160.73,160.73,11813.83,6,6,191.00,30.00
172089,2021-03-12,Kern,California,104763,982.00,6029,CA,County,900202,11637.72,109.09,109.09,11637.72,14,7,199.00,18.00
435441,2021-03-12,Merced,California,29796,421.00,6047,CA,County,277680,10730.34,151.61,151.61,10730.34,10,8,85.00,3.00
61325,2021-03-12,Stanislaus,California,57373,966.00,6099,CA,County,550660,10418.95,175.43,175.43,10418.95,4,9,153.00,3.00
76002,2021-03-12,Tulare,California,48550,784.00,6107,CA,County,466195,10414.10,168.17,168.17,10414.10,5,10,71.00,2.00


In [17]:
# Show top 10 California counties based on largest current death rate

counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)
top10_ca_death_rates = counties_cal.head(10)
top10_ca_death_rates

<ipython-input-17-018d0b919d54>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  counties_cal.sort_values(by=['drate'], inplace=True, ascending=False)


,date,county,state,cases,deaths,FIPStxt,State2,Region,population,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
274613,2021-03-12,Imperial,California,27045,677.00,6025,CA,County,181215,14924.26,373.59,373.59,14924.26,1,2,-6.00,2.00
2067,2021-03-12,Los Angeles,California,1208678,22304.00,6037,CA,County,10039107,12039.70,222.17,222.17,12039.70,2,5,6.00,0.00
480382,2021-03-12,Inyo,California,1335,37.00,6027,CA,County,18039,7400.63,205.11,205.11,7400.63,3,25,7.00,0.00
61325,2021-03-12,Stanislaus,California,57373,966.00,6099,CA,County,550660,10418.95,175.43,175.43,10418.95,4,9,153.00,3.00
76002,2021-03-12,Tulare,California,48550,784.00,6107,CA,County,466195,10414.10,168.17,168.17,10414.10,5,10,71.00,2.00
28467,2021-03-12,Riverside,California,291866,3971.00,6065,CA,County,2470546,11813.83,160.73,160.73,11813.83,6,6,191.00,30.00
48815,2021-03-12,San Joaquin,California,67817,1210.00,6077,CA,County,762148,8898.14,158.76,158.76,8898.14,7,18,0.00,0.00
582260,2021-03-12,Kings,California,22320,235.00,6031,CA,County,152940,14593.96,153.66,153.66,14593.96,8,3,50.00,2.00
27725,2021-03-12,Fresno,California,97026,1527.00,6019,CA,County,999101,9711.33,152.84,152.84,9711.33,9,14,160.00,8.00
435441,2021-03-12,Merced,California,29796,421.00,6047,CA,County,277680,10730.34,151.61,151.61,10730.34,10,8,85.00,3.00


In [18]:
# Create dataframe with historical data for Top 10 counties based on current case rate

top10c_df = top10_ca_case_rates[['county', 'state']]
top10_ca_case_df = pd.merge(usa, top10c_df, on=['county', 'state'])
top10_ca_case_df['date'] = pd.to_datetime(top10_ca_case_df['date']).dt.strftime('%m/%d/%y')
top10_ca_case_df = top10_ca_case_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_case_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,373.59,14924.26,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,373.59,14924.26,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,373.59,14924.26,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,373.59,14924.26,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,373.59,14924.26,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3592,03/08/21,Lassen,5629,25.00,18411.67,81.77,78.50,18411.67,31,1,6.00,1.00
3593,03/09/21,Lassen,5629,25.00,18411.67,81.77,78.50,18411.67,31,1,0.00,0.00
3594,03/10/21,Lassen,5629,24.00,18411.67,78.50,78.50,18411.67,31,1,0.00,-1.00
3595,03/11/21,Lassen,5629,24.00,18411.67,78.50,78.50,18411.67,31,1,0.00,0.00


In [19]:
# Create dataframe with historical data for Top 10 counties based on current death rate

top10d_df = top10_ca_death_rates[['county', 'state']]
top10_ca_death_df = pd.merge(usa, top10d_df, on=['county', 'state'])
top10_ca_death_df['date'] = pd.to_datetime(top10_ca_death_df['date']).dt.strftime('%m/%d/%y')
top10_ca_death_df = top10_ca_death_df[['date', 'county','cases', 'deaths', 'current_crate', 'current_drate', 'drate', 'crate', 'state_death', 'state_case', 'case_day', 'death_day']]
top10_ca_death_df

,date,county,cases,deaths,current_crate,current_drate,drate,crate,state_death,state_case,case_day,death_day
0,03/20/20,Imperial,4,0.00,2.21,0.00,373.59,14924.26,1,2,4.00,0.00
1,03/21/20,Imperial,4,0.00,2.21,0.00,373.59,14924.26,1,2,0.00,0.00
2,03/22/20,Imperial,4,0.00,2.21,0.00,373.59,14924.26,1,2,0.00,0.00
3,03/23/20,Imperial,4,0.00,2.21,0.00,373.59,14924.26,1,2,0.00,0.00
4,03/24/20,Imperial,9,0.00,4.97,0.00,373.59,14924.26,1,2,5.00,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...
3666,03/08/21,Merced,29557,415.00,10644.27,149.45,151.61,10730.34,10,8,181.00,2.00
3667,03/09/21,Merced,29600,416.00,10659.75,149.81,151.61,10730.34,10,8,43.00,1.00
3668,03/10/21,Merced,29643,417.00,10675.24,150.17,151.61,10730.34,10,8,43.00,1.00
3669,03/11/21,Merced,29711,418.00,10699.73,150.53,151.61,10730.34,10,8,68.00,1.00


In [20]:
# Export California Top 10 case data to csv

top10_ca_case_df.to_csv('top10cases.csv', float_format='%.2f', index=False)

In [21]:
# Export California Top 10 death data to csv

top10_ca_death_df.to_csv('top10deaths.csv', float_format='%.2f', index=False)